# Ejercicio Formativo 1 Capítulo 7

## Importando librerías

In [1]:
import sqlite3

## Revisando la base de datos

Antes de empezar con el desarrollo de las consultas es recomendable revisar la base de datos para conocer las tablas y columnas con las que se va a trabajar. Para ello, se puede reutilizar el código visto en clase:

In [2]:
connection = sqlite3.connect('sales.db')
cursor = connection.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tablas:\n")
for table in cursor.fetchall():
    print(table[0])
    cursor.execute(f'PRAGMA table_info([{table[0]}])')
    print(cursor.fetchall())
    print()
connection.close()

Tablas:

Proveedores
[(0, 'pid', 'INTEGER', 0, None, 1), (1, 'nombre', 'TEXT', 0, None, 0), (2, 'direccion', 'TEXT', 0, None, 0)]

Partes
[(0, 'paid', 'INTEGER', 0, None, 1), (1, 'nombre', 'INTEGER', 0, None, 0), (2, 'color', 'TEXT', 0, None, 0)]

Catalogo
[(0, 'pid', 'INTEGER', 0, None, 1), (1, 'paid', 'INTEGER', 0, None, 2), (2, 'valor', 'REAL', 0, None, 0)]



In [3]:
connection = sqlite3.connect('sales.db')
cursor = connection.cursor()

In [4]:
sql = "SELECT * FROM Proveedores"
cursor.execute(sql)
for r in cursor:
    print(r)

(1, 'Acme Widget Suppliers', '1 Grub St., Potemkin Village, IL 61801')
(2, 'Big Red Tool and Die', '4 My Way, Bermuda Shorts, OR 90305')
(3, 'Perfunctory Parts', '99999 Short Pier, Terra Del Fuego, TX 41299')
(4, 'Alien Aircaft Inc.', '2 Groom Lake, Rachel, NV 51902')


In [5]:
sql = "SELECT * FROM Partes"
cursor.execute(sql)
for r in cursor:
    print(r)

(1, 'Left Handed Bacon Stretcher Cover', 'Red')
(2, 'Smoke Shifter End', 'Black')
(3, 'Acme Widget Washer', 'Red')
(4, 'Acme Widget Washer', 'Silver')
(5, 'I Brake for Crop Circles Sticker', 'Translucent')
(6, 'Anti-Gravity Turbine Generator', 'Cyan')
(7, 'Anti-Gravity Turbine Generator', 'Magenta')
(8, 'Fire Hydrant Cap', 'Red')
(9, '7 Segment Display', 'Green')


In [6]:
sql = "SELECT * FROM Catalogo"
cursor.execute(sql)
for r in cursor:
    print(r)

(1, 3, 0.5)
(1, 4, 0.5)
(1, 8, 11.7)
(2, 3, 0.55)
(2, 8, 7.95)
(2, 1, 16.5)
(3, 8, 12.5)
(3, 9, 1.0)
(4, 5, 2.2)
(4, 6, 1247548.23)
(4, 7, 1247548.23)


In [7]:
connection.close()

## Consultas

### Consulta 1: Encuentre los nombres de los proveedores que que venden todas las partes que son de color rojo

In [8]:
connection = sqlite3.connect("sales.db")
cursor = connection.cursor()
cursor.execute(
    "SELECT DISTINCT P.nombre \
      FROM Proveedores P, Partes R, Catalogo C \
      WHERE P.pid = C.pid AND R.paid = C.paid AND R.color = 'Red'"
)
for r in cursor:
    print(r[0])
connection.close()

Acme Widget Suppliers
Big Red Tool and Die
Perfunctory Parts


#### Explicación

En esta consulta nos solicitan encontrar los nombres de los proveedores que venden todas las partes que son de color rojo. Para ello, se realizan consultas entre tablas (JOIN), para Proveedores, Partes y Catalogo, haciendo coincidir sus identificadores y filtrando por el color de las partes, en este caso, rojo. Además, dado que un mismo proveedor puede vender varias partes, se señala que los nombres de estos proveedores deben aparecer una sola vez en la respuesta, por lo que se utiliza la cláusula DISTINCT.

### Consulta 2: Encuentre los nombres de las partes que son solo vendidas por una empresa cuyo nombre contiene el string ACME.

In [9]:
connection = sqlite3.connect("sales.db")
cursor = connection.cursor()
cursor.execute(
    "SELECT DISTINCT R.paid, R.nombre, R.color \
      FROM Proveedores P, Partes R, Catalogo C \
      WHERE P.pid = C.pid AND R.paid = C.paid AND P.nombre LIKE '%ACME%' \
      AND R.paid NOT IN (SELECT R2.paid \
      FROM Proveedores P2, Partes R2, Catalogo C2 \
      WHERE P2.pid = C2.pid AND R2.paid = C2.paid AND P2.nombre NOT LIKE '%ACME%')"
)
for r in cursor:
    print(r)
connection.close()

(4, 'Acme Widget Washer', 'Silver')


#### Explicación

Al igual que para la consulta anterior se utilizan consultas entre tablas (JOIN), para Proveedores, Partes y Catalogo, haciendo coincidir sus identificadores. En este caso, se filtra por el nombre de la empresa proveedora, buscando aquellas que contengan el string ACME. Esto se logra hasta esta parte de la consulta:

```sql
SELECT DISTINCT R.paid, R.nombre, R.color \
FROM Proveedores P, Partes R, Catalogo C \
WHERE P.pid = C.pid AND R.paid = C.paid AND P.nombre LIKE '%ACME%'
```
Sin embargo, nos indican que se deben encontrar las partes que son solo vendidas por una empresa cuyo nombre contiene el string ACME. Para ello, se debe verificar que no existan otras empresas que vendan la misma parte y que no contengan el string ACME en su nombre. Esto se logra con la siguiente parte de la consulta:

```sql
... AND R.paid NOT IN (SELECT R2.paid \
    FROM Proveedores P2, Partes R2, Catalogo C2 \
    WHERE P2.pid = C2.pid AND R2.paid = C2.paid AND P2.nombre NOT LIKE '%ACME%')
```

Se imprimen más datos de los solicitados, para que se puede diferenciar a que parte corresponde ya que hay dos partes con el mismo nombre.

### Consulta 3: Encuentre los nombres de los proveedores que cobran por alguna parte más que el precio promedio de esta (calculado sobre todos los proveedores que la venden).

In [20]:
connection = sqlite3.connect("sales.db")
cursor = connection.cursor()
cursor.execute(
    "SELECT P.nombre FROM Proveedores P, Partes R, Catalogo C \
        WHERE P.pid = C.pid AND R.paid = C.paid AND \
        C.valor > (SELECT AVG(C2.valor)\
      FROM Partes R2, Catalogo C2 \
      WHERE R2.paid = C2.paid AND R.paid = R2.paid\
      GROUP BY R2.paid)"
)
for r in cursor:
    print(r[0])
connection.close()

Acme Widget Suppliers
Big Red Tool and Die
Perfunctory Parts


#### Explicación

Esta consulta viene siendo ya un poco más compleja, ya que se requiere calcular el precio promedio de cada parte y compararlo con el precio de cada parte vendida por los proveedores. Para ello, se pueden utilizar subconsultas, en este caso lo que se hizo fue calcular el precio promedio de cada parte, a fin de exlicar mejor la consulta, veamos que nos devuelve esta subconsulta por si sola:

In [16]:
connection = sqlite3.connect("sales.db")
cursor = connection.cursor()
cursor.execute(
    "SELECT R2.paid, R2.nombre, AVG(C2.valor)\
      FROM Partes R2, Catalogo C2 \
      WHERE R2.paid = C2.paid \
      GROUP BY R2.paid"
)
for r in cursor:
    print(r)
connection.close()

(1, 'Left Handed Bacon Stretcher Cover', 16.5)
(3, 'Acme Widget Washer', 0.525)
(4, 'Acme Widget Washer', 0.5)
(5, 'I Brake for Crop Circles Sticker', 2.2)
(6, 'Anti-Gravity Turbine Generator', 1247548.23)
(7, 'Anti-Gravity Turbine Generator', 1247548.23)
(8, 'Fire Hydrant Cap', 10.716666666666667)
(9, '7 Segment Display', 1.0)


Como se puede ver lo que nos devuelve es el precio promedio de cada parte, y luego se compara con el precio de cada parte vendida por los proveedores, para ver cual de ellos supera el precio promedio de la parte, esto se logra al hacer la siguiente comparación:

```sql
... AND \
C.valor > (SELECT AVG(C2.valor)\
FROM Partes R2, Catalogo C2 \
WHERE R2.paid = C2.paid AND R.paid = R2.paid\
GROUP BY R2.paid)
```

En esta parte se indica que se debe cumplir que el valor asociado a la parte debe superar el promedio de los valores de la parte, calculado sobre todos los proveedores que la venden, para asegurarse que se refiere a la misma parte se hace la comparación de los identificadores de las partes entre las consulta principal y la subconsulta, esto:
    
```sql
R.paid = R2.paid
```

#### Cláusula WITH

Un punto a señalar es que algunas veces las consultas terminar siendo muy largas y complejas, por lo que uno mismo puede terminarse perdiento en su propio código, para evitar esto se puede utilizar la cláusula WITH, que permite definir subconsultas que se pueden reutilizar en la consulta principal, esto hace que el código sea más legible y fácil de entender, por ejemplo la consulta 3 se puede reescribir de la siguiente manera:

In [21]:
connection = sqlite3.connect("sales.db")
cursor = connection.cursor()
cursor.execute(
    "WITH PromedioPartes AS \
      (SELECT R.paid as paid, AVG(C.valor) as  avg_valor\
      FROM Partes R, Catalogo C \
      WHERE R.paid = C.paid \
      GROUP BY R.paid) \
          SELECT P.nombre FROM Proveedores P, Partes R, Catalogo C, PromedioPartes PP \
      WHERE P.pid = C.pid AND R.paid = C.paid AND  R.paid = PP.paid AND C.valor > PP.avg_valor"
)
for r in cursor:
    print(r[0])
connection.close()

Acme Widget Suppliers
Big Red Tool and Die
Perfunctory Parts


Como pueden ver se obtuvo el mismo resultado, ahora descompgamos un poco esta nueva consulta. En primer luagr, la manera de usar esta claúsula es la siguiente:

```sql
WITH NombreSubconsulta AS (
    SELECT ...
    FROM ...
    WHERE ...
)
SELECT ...
FROM ...
WHERE ...
```
En la parte de *NombreSubconsulta* se debe colocar el nombre que se le quiere dar a la subconsulta, y para utilizarla se usa la misma sintaxis si estuvieramos utilizando una tabla ya creada en la base de datos. Ahora veamoslo a más detalle para la consulta 3:

```sql
WITH PromedioPartes AS \
      (SELECT R.paid as paid, AVG(C.valor) as  avg_valor\
      FROM Partes R, Catalogo C \
      WHERE R.paid = C.paid \
      GROUP BY R.paid)\
```
En esta parte se calcula el precio promedio de cada parte, y se le asigna el nombre de *PromedioPartes*, que es la misma subconsulta que se había hecho anteriormente. Luego se utiliza esta subconsulta en la consulta principal de la siguiente manera:

```sql
... SELECT P.nombre FROM Proveedores P, Partes R, Catalogo C, PromedioPartes PP \
      WHERE P.pid = C.pid AND R.paid = C.paid AND  R.paid = PP.paid AND C.valor > PP.avg_valor
```
Donde se realiza JOIN entre las tablas Proveedores, Partes, Catalogo y la subconsulta PromedioPartes, y se filtra por los proveedores que venden partes cuyo valor supera el promedio de las partes.

### Consulta 4: Para cada proveedor que vende partes verdes y rojas, encuentre el nombre y precio de la parte más cara que vende.

In [23]:
connection = sqlite3.connect("sales.db")
cursor = connection.cursor()
cursor.execute(
    "SELECT P.nombre, R.nombre, MAX(C.valor) FROM Proveedores P, Partes R, Catalogo C WHERE P.pid = C.pid AND R.paid = C.paid \
      AND P.pid IN (SELECT P.pid FROM Proveedores P, Partes R, Catalogo C \
      WHERE P.pid = C.pid AND R.paid = C.paid AND R.color = 'Green') \
      AND P.pid IN (SELECT P.pid FROM Proveedores P, Partes R, Catalogo C \
      WHERE P.pid = C.pid AND R.paid = C.paid AND R.color = 'Red') GROUP BY P.pid"
)
for r in cursor:
    print(r)
connection.close()

('Perfunctory Parts', 'Fire Hydrant Cap', 12.5)


#### Explicación

Para esta consulta en primer lugar se debe determinar que proveedores venden partes verdes y rojas, para ello se puede utilizar la siguiente consulta:

```sql
SELECT P.pid FROM Proveedores P, Partes R, Catalogo C \
      WHERE P.pid = C.pid AND R.paid = C.paid AND R.color = COLOR
```
Un punto importante a señalar es que el enunciado nos solicita que el vendedor venda tanto partes verdes como rojas por lo cual se hizo una subconsulta para cada color y luego se buscaba los proveedores que estuvieran en ambas subconsultas. Ahora armando la consulta principal, lo que se hace es hacer JOIN de las tablas Proveedores, Partes y Catalogo, y se filtra por los proveedores que venden partes verdes y rojas, esto haciendo:

```sql
...AND P.pid IN (SELECT P.pid FROM Proveedores P, Partes R, Catalogo C \
      WHERE P.pid = C.pid AND R.paid = C.paid AND R.color = 'Green') \
      AND P.pid IN (SELECT P.pid FROM Proveedores P, Partes R, Catalogo C \
      WHERE P.pid = C.pid AND R.paid = C.paid AND R.color = 'Red')
```
Luego como se pide encontrar el nombre y precio de la parte más cara que vende cada proveedor, para ello se agrupa por el identificador del proveedor y se utiliza la función MAX para encontrar el precio más alto, esto se logra con la siguiente parte en la consulta principal:

```sql
SELECT R.nombre, MAX(C.valor) FROM Proveedores P, Partes R, Catalogo C WHERE P.pid = C.pid AND R.paid = C.paid \
... (Verifico que el proveedor venda partes verdes y rojas) \
GROUP BY P.pid
```
